# Langchain & ChromaDB

In [ ]:
!pip install --upgrade --quiet  langchain-google-genai
!pip install PyPDF2
!pip install langchain
!pip install chromadb
! pip install OpenAI ## Will Be using this one In this
! pip install unstructured
!pip install tiktoken


In [ ]:
import langchain
import getpass
import os
import pandas as pd
from PyPDF2 import PdfReader
from langchain.llms import OpenAI
from langchain.embeddings import OpenAIEmbeddings
import google.generativeai as genai1
from langchain_google_genai import GoogleGenerativeAIEmbeddings as ge
from langchain_google_genai import ChatGoogleGenerativeAI as genai
from langchain.chains import LLMChain,ConversationChain
from langchain.chains.conversation.memory import (
    ConversationBufferMemory,
    ConversationSummaryBufferMemory,
    ConversationKGMemory,
    ConversationSummaryMemory
)
  # Import a PDF library
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma

In [ ]:
if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Provide your Google API key here")

Provide your Google API key here··········


In [ ]:
def pdf_to_text(pdf_path):
    with open(pdf_path,'rb') as file:
        docs=PdfReader(file)
        text=''
        num_pages=len(docs.pages)
        for page_num in range(num_pages):
          page=docs.pages[page_num]
          text+=page.extract_text()
    return text

In [ ]:
def text_splitter(text,chunk_size:int=2000,chunk_overlap:int=500):
  t_splitter=RecursiveCharacterTextSplitter(chunk_size=chunk_size,chunk_overlap=chunk_overlap)
  return t_splitter.split_documents(text)

In [ ]:
def write_file(file_name,text):
  with open(file_name,'w') as file:
    file.write(text)


In [ ]:
text=pdf_to_text('LoRA_Paper.pdf')

In [ ]:
write_file('Lora.txt',text)

In [ ]:
loader=DirectoryLoader('/content',glob='./*.txt')

In [ ]:
documnets=loader.load()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [ ]:
chunks=text_splitter(documnets)

In [ ]:
chunks[0]

Document(page_content='LORA: L OW-RANK ADAPTATION OF LARGE LAN-\n\nGUAGE MODELS\n\nEdward Hu\x03Yelong Shen\x03Phillip Wallis Zeyuan Allen-Zhu\n\nYuanzhi Li Shean Wang Lu Wang Weizhu Chen\n\nMicrosoft Corporation\n\nfedwardhu, yeshe, phwallis, zeyuana,\n\nyuanzhil, swang, luw, wzchen g@microsoft.com\n\nyuanzhil@andrew.cmu.edu\n\n(Version 2)\n\nABSTRACT\n\nAn important paradigm of natural language processing consists of large-scale pre-\n\ntraining on general domain data and adaptation to particular tasks or domains. As\n\nwe pre-train larger models, full ﬁne-tuning, which retrains all model parameters,\n\nbecomes less feasible. Using GPT-3 175B as an example – deploying indepen-\n\ndent instances of ﬁne-tuned models, each with 175B parameters, is prohibitively\n\nexpensive. We propose Low-RankAdaptation, or LoRA, which freezes the pre-\n\ntrained model weights and injects trainable rank decomposition matrices into each\n\nlayer of the Transformer architecture, greatly reducing the numb

In [ ]:
embedding=OpenAIEmbeddings()

In [ ]:
persist_directory='db'

In [ ]:
vectordb=Chroma.from_documents(documents=chunks,embedding=embedding,persist_directory=persist_directory)

In [ ]:
vectordb.persist()
vectordb=None

In [ ]:
vectordb=Chroma(persist_directory=persist_directory,embedding_function=embedding)

In [ ]:
retriever=vectordb.as_retriever(search_kwargs={'k':3})

In [ ]:
docs=retriever.get_relevant_documents('Why QLora is better')

In [ ]:
len(docs)

3

In [ ]:
qa_chain=RetrievalQA.from_chain_type(llm=OpenAI(),chain_type='stuff',retriever=retriever,return_source_documents=True)

In [ ]:
res=qa_chain('How does QLaRA Work ')

In [ ]:
res

{'query': 'How does QLaRA Work ',
 'result': ' LoRA works by reducing the storage and memory requirements for training models by adapting only a subset of weight matrices, rather than the entire model. This allows for more efficient training and lower hardware barriers to entry. Additionally, LoRA allows for easier task-switching and the creation of customized models that can be swapped in and out on the fly. It also provides a 25% speedup during training compared to full fine-tuning.',
 'source_documents': [Document(page_content='ferent tasks. We can freeze the shared model and efﬁciently switch tasks by replacing the\n\nmatricesAandBin Figure 1, reducing the storage requirement and task-switching over-\n\nhead signiﬁcantly.\n\nLoRA makes training more efﬁcient and lowers the hardware barrier to entry by up to 3\n\ntimes when using adaptive optimizers since we do not need to calculate the gradients or\n\nmaintain the optimizer states for most parameters. Instead, we only optimize the 

In [ ]:
import pprint

In [ ]:
def clear_res(llm_response):
  user_query=llm_response['query']
  llm_res=llm_response['result']
  print(f"{user_query}")
  print(f"\n{llm_res}")
  print('\n\n')
  source=[]
  for _ in llm_response['source_documents']:
    print(_.metadata['source'])

In [ ]:
clear_res(res)

How does QLaRA Work 

 LoRA works by reducing the storage and memory requirements for training models by adapting only a subset of weight matrices, rather than the entire model. This allows for more efficient training and lower hardware barriers to entry. Additionally, LoRA allows for easier task-switching and the creation of customized models that can be swapped in and out on the fly. It also provides a 25% speedup during training compared to full fine-tuning.



/content/Lora.txt
/content/Lora.txt
/content/Lora.txt
